In [1]:
import sys
!pip install pandas
!pip install numpy
!pip install datapane
!pip install plotly
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
  Created wheel for micawber: filename=micawber-0.5.5-py3-none-any.whl size=22434 sha256=7346dbf234251a0c95e15fb156d306097e5ed6d708917980a32b822fc61b5821
  Stored in directory: /root/.cache/pip/wheels/31/3a/1f/bbd88c767423821f2fe497c0ada47581292f96419b56d689c0
Successfully built micawber
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.6 MB/s eta 0:00:00


In [ ]:
# import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datapane as dp
import pymysql
import warnings
import pymysql
import sys
import os

warnings.filterwarnings('ignore')



In [ ]:
host = 'database-2.cd1ssa01itfp.us-east-2.rds.amazonaws.com'
user = 'admin'
password = '1Mankiran'
database = 'VitalRingService'

connection = pymysql.connect(host=host, user=user, passwd=password, database=database)
with connection:
    cur = connection.cursor()
    cur.execute("SELECT VERSION()")
    version = cur.fetchone()
    print("Database version: {} ".format(version[0]))

Database version: 8.0.33 


In [ ]:

def run_query(q: str) -> pd.DataFrame:
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        return pd.read_sql(q, conn)


def run_command(c: str):
    with pymysql.connect(host=host, user=user, passwd=password, database=database) as conn:
        conn.isolation_level = None
        conn.execute(c)

# Entities Count

In [ ]:
def show_tables() -> pd.DataFrame:
    q = """
          SELECT table_name
          FROM information_schema.tables
          WHERE table_schema = 'VitalRingService' AND table_type = 'BASE TABLE';
        """
    return run_query(q)


def get_table_row_count(tablename) -> int:
    q = (
        """
        SELECT
            COUNT(1)
        FROM %s;
        """
        % tablename
    )
    return run_query(q)["COUNT(1)"][0]


tables = show_tables()
tables["row_count"] = [get_table_row_count(t) for t in tables['TABLE_NAME']]

tables

,TABLE_NAME,row_count
0,Address,17
1,Agent,8
2,Article,3
3,CallLog,11
4,CaseArticle,6
5,Customer,10
6,Feedback,10
7,OrderLine,12
8,Product,6
9,ServiceOrder,15


In [ ]:
#query1

query_1 = """
SELECT 'Agent' AS TableName, COUNT(*) AS RowCount FROM Agent
UNION ALL
SELECT 'Feedback', COUNT(*) FROM Feedback
UNION ALL
SELECT 'Customer', COUNT(*) FROM Customer
UNION ALL
SELECT 'Address', COUNT(*) FROM Address
UNION ALL
SELECT 'ServiceRequest', COUNT(*) FROM ServiceRequest
UNION ALL
SELECT 'ServiceOrder', COUNT(*) FROM ServiceOrder
UNION ALL
SELECT 'OrderLine', COUNT(*) FROM OrderLine
UNION ALL
SELECT 'Product', COUNT(*) FROM Product
UNION ALL
SELECT 'Article', COUNT(*) FROM Article
UNION ALL
SELECT 'CaseArticle', COUNT(*) FROM CaseArticle
UNION ALL
SELECT 'CallLog', COUNT(*) FROM CallLog;

"""

query1 = run_query(query_1)

dp.DataTable(query1)

# Assuming run_query returns a DataFrame
df = run_query(query_1)

# First, sort the DataFrame by 'CustomerID' in ascending order
df_sorted = df.sort_values(by='TableName', ascending=True)

# Define the colors you want to use for the bars
colors = ['black', 'blue', 'purple', 'red', 'black']

# Create an interactive bar chart
fig_1 = px.bar(df_sorted, x='TableName', y='RowCount',
             title='Raw Counts of all Tables',
             labels={'TableName': 'Table Name', 'RowCount': 'Row Count'},
             color_discrete_sequence=colors)  # Set the colors

# Show the figure
fig_1.show()

# Articles Count By Case Usage

In [ ]:
import datapane as dp

top_10_tracks_query = """
SELECT
    a.ArticleID as articleId, a.Content as content, count(*) as usage_count
FROM CaseArticle ca, Article a, ServiceRequest sr
WHERE ca.CaseID = sr.CaseID and
ca.ArticleID = a.ArticleID
GROUP BY ca.ArticleId having count(*) > 1
"""

top_10_df = run_query(top_10_tracks_query)

dp.DataTable(top_10_df)

##Customers by Cases

Displays the cases that customers have placed

In [ ]:
customers_to_case = """
WITH
    usa_tracks_sold AS
        (
         SELECT sr.* FROM Customer c
         INNER JOIN ServiceRequest sr ON sr.CustomerID = c.CustomerID
        )
SELECT
    CONCAT(c.FirstName,'',c.LastName) Name,
    COUNT(uts.CustomerId) "Number of Customer",
    CAST(COUNT(uts.CustomerId) AS FLOAT)/(SELECT COUNT(*) FROM usa_tracks_sold) "Percentage of Cases"
FROM usa_tracks_sold uts
INNER JOIN Customer c ON uts.CustomerID = c.CustomerID
GROUP BY 1
LIMIT 10;
"""

genre_sales = run_query(customers_to_case)
genre_sales.sort_values("Number of Customer", inplace=True)

customer_case_plot = px.bar(
    genre_sales,
    x="Percentage of Cases",
    y="Name",
    orientation="h",
    text="Percentage of Cases",
    title="Cases by Customer",
)

customer_case_plot.update_traces(texttemplate="%{text:.1%}", textposition="outside")

customer_case_plot.show()

In [ ]:
import datapane as dp

query_10 = """
SELECT
    Cust.CustomerID, Cust.FirstName, Cust.LastName,
    COUNT(DISTINCT sr.CaseID) AS NumberOfCases,
    COUNT(DISTINCT cl.CallLogID) AS NumberOfCalls,
    AVG(Fdbk.SatisfactionScore) AS AverageSatisfactionScore
FROM
    Customer AS Cust
LEFT JOIN ServiceRequest AS sr ON Cust.CustomerID = sr.CustomerID
LEFT JOIN CallLog AS cl ON sr.CaseID = cl.CaseID
LEFT JOIN Feedback AS Fdbk ON Cust.CustomerID = Fdbk.CustomerID
GROUP BY
    Cust.CustomerID, Cust.FirstName, Cust.LastName;

"""

query10 = run_query(query_10)

dp.DataTable(query10)

In [ ]:
# Run SQL query to get the data
customers_with_case_count = """
SELECT
    Cust.CustomerID,
    Cust.FirstName,
    Cust.LastName,
    COUNT(sr.CaseID) AS CaseCount
FROM
    Customer Cust
LEFT JOIN ServiceRequest sr ON Cust.CustomerID = sr.CustomerID
GROUP BY
    Cust.CustomerID, Cust.FirstName, Cust.LastName
ORDER BY
    Cust.CustomerID;

"""

# Assuming run_query returns a DataFrame
df = run_query(customers_with_case_count)

# First, sort the DataFrame by 'CustomerID' in ascending order
df_sorted = df.sort_values(by='CustomerID', ascending=True)

# Define the colors you want to use for the bars
colors = ['orange', 'blue', 'purple', 'red', 'black']

# Create an interactive bar chart
fig = px.bar(df_sorted, x='CustomerID', y='CaseCount',
             title='Case Count per Customer',
             labels={'CustomerID': 'Customer ID', 'CaseCount': 'Case Count'},
             color_discrete_sequence=colors)  # Set the colors

# Show the figure

fig.show()

In [ ]:
#query5: Call log count by cases

import datapane as dp

call_log_count_by_cases = """
SELECT sr.CaseID, COUNT(cl.CallLogID) AS CallLogCount
FROM ServiceRequest sr
JOIN CallLog cl ON sr.CaseID = cl.CaseID
GROUP BY sr.CaseID
ORDER BY CallLogCount DESC;

"""

query5 = run_query(call_log_count_by_cases)

dp.DataTable(query5)

# Agent Performance

We have 2 managers and each of them are managing a team of agents. Our question is: "Which team performs better?"
We created a donut chart to visualize this question. The third group includes cases handled by managers themselves. The visualization shows that Managers handled most of the cases themselves followed by Will Smith's team. Mary Wilson's team solved 13% of all cases

In [ ]:
manager_performance = """
SELECT COALESCE(CONCAT( Manager.FirstName,' ', Manager.LastName, '''s team'),'Handled by manager') AS Team, SUM(t.NumberOfCases) AS NumberOfCases
FROM Agent
LEFT JOIN
(
SELECT AgentID, COUNT(CaseID) AS NumberOfCases
FROM ServiceRequest
GROUP BY AgentID
) AS t ON Agent.AgentID=t.AgentID
Left Join Agent as Manager On Agent.ManagerID=Manager.AgentID
GROUP BY 1

"""


run_query(manager_performance)
df = run_query(manager_performance)

sales_breakdown = go.Figure(
    data=[go.Pie(labels=df["Team"], values=df["NumberOfCases"], hole=0.3)]
)

sales_breakdown.update_layout(title_text="Team Performance")

sales_breakdown.show()

Here we want to know which agent handled the most cases. So we created a bar chart with all the agents and their corresponding number of cases.

In [ ]:
agent_performance = """
SELECT Agent.AgentID, FirstName, LastName, t.NumberOfCases
FROM Agent
JOIN
(
SELECT AgentID, COUNT(CaseID) AS NumberOfCases
FROM ServiceRequest
GROUP BY AgentID
#Limit 1
) AS t ON Agent.AgentID=t.AgentID
ORDER BY t.NumberOfCases Desc
"""

df = run_query(agent_performance)
df.sort_values("NumberOfCases", inplace=True)

# Define the colors you want to use for the bars
colors = ['purple', 'blue', 'purple', 'red', 'black']

performance_plot = px.bar(
    df,
    x="NumberOfCases",
    y='FirstName',
    orientation="h",
    text="NumberOfCases",
    title="Agent Performance",
    color_discrete_sequence=colors
)

performance_plot.update_traces(texttemplate="%{text:}", textposition="outside")

performance_plot.show()

# Putting it into a Datapane report

Adding all the reports to this dashboard

In [ ]:
customers = tables[tables["TABLE_NAME"] == "Customer"]["row_count"].values[0]
agents = tables[tables["TABLE_NAME"] == "Agent"]["row_count"].values[0]
cases_submitted = tables[tables["TABLE_NAME"] == "ServiceRequest"]["row_count"].values[0]
orders_placed = tables[tables["TABLE_NAME"] == "ServiceOrder"]["row_count"].values[0]

In [ ]:
r = dp.View(
    dp.Group(
        dp.BigNumber(heading="Total customers", value=customers),
        dp.BigNumber(heading="Total agents", value=agents),
        dp.BigNumber(heading="Total cases submitted", value=cases_submitted),
        dp.BigNumber(heading="Total orders placed", value=orders_placed),
        columns=4,
        name="Little_group",
    ),
    dp.Plot(fig_1, name="entities_count"),
    dp.Plot(fig, name="case_count_by_customer"),
    dp.Plot(customer_case_plot, name="customers_by_case"),
    dp.Plot(sales_breakdown, name="Team_Performance"),
    dp.Plot(performance_plot, name="agent_performance"),
    dp.DataTable(top_10_df, name="customer-cases"),
    dp.DataTable(query10), name="customer_metrics"


)
dp.save_report(r, "report.html", open=True)

App saved to ./report.html

In [ ]:
import IPython
IPython.display.HTML(filename='report.html')